In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC, LinearSVC
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack, vstack, csc_matrix
from nltk.stem import PorterStemmer
import scipy as sc
from sklearn.preprocessing import OneHotEncoder
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import nltk
import re
from scipy.sparse import hstack, csr_matrix
%matplotlib inline

In [2]:
def base_and_ending(X, y):
    base = []
    ending = []
    flag = True
    if len(X) >= len(y):
        length = len(y)
    else:
        length = len(X)
    for i in range(length):
        if X[i] == y[i] and flag==True:
            base.append(X[i])
        else:
            flag = False
    ending = y.replace(''.join(base),"")
    return ''.join(base), ending

In [3]:
def vowel_or_consonant(word):
    alpha = word[-1] 
    if alpha == 'a' or alpha == 'e' or alpha == 'i' or alpha == 'o' or alpha =='u' or alpha == 'y':
        return 1
    else:
        return 0

In [4]:
def ratio_v_and_c(word):
    v, c = 0., 0.
    for alpha in word:
        if alpha == 'a' or alpha == 'e' or alpha == 'i' or alpha == 'o' or alpha =='u' or alpha == 'y':
            v += 1
        else:
            c += 1
    if c == 0:
        return 0
    else:
        return v/c        

In [5]:
def transform_features(data):    
    data = data.drop(['Id'], axis=1)
    data = data.drop(0, axis=0)    
    
    form = []
    for i in data.y:
        form.append(i[-1])
    data['form'] = form
    
    data['word'] = data.y.map(lambda x: re.sub("[+,V,A,N]", "", x))
    
    base = []
    ending = []
    cv_ratio = []
    length = []
    for X, y in zip(data.X, data.word):
        cv_ratio.append(ratio_v_and_c(X))
        length.append(len(X))
        b, e = base_and_ending(X, y)
        base.append(b)
        ending.append(e)
    data['base'] = base
    data['ending'] = ending
    data['ratio'] = cv_ratio
    data['X_length'] = length
    
    end_length = []
    for X, base in zip(data.X, data.base):        
        end_length.append(len(X.replace(base,"")))
    data['end_l'] = end_length
    data = data.drop(['y'], axis=1)
    return data

In [29]:
train = pd.read_csv('task2_lemmas_train.txt', names=['Id', 'X', 'y'])
test = pd.read_csv('task2_lemmas_test.txt', names=['Id', 'X'])
sample = pd.read_csv('task2_lemmas_sample_submission.txt', names=['Id', 'Category'])

train = transform_features(train)

test = test.drop(['Id'], axis=1)
test = test.drop(0, axis=0) 

sample = sample.drop(['Id'], axis=1)
sample = sample.drop(0, axis=0)

In [30]:
train[:3]

,X,form,word,base,ending,ratio,X_length,end_l
1,vergognerete,V,vergognare,vergogn,are,0.714286,12,5
2,amnistiavate,V,amnistiare,amnistia,re,1.000000,12,4
3,menomazione,N,menomazione,menomazione,,1.200000,11,0


In [31]:
test_cv_ratio, test_length = [], []
for X in test.X:
        test_cv_ratio.append(ratio_v_and_c(X))
        test_length.append(len(X))
test['X_length'] = test_length
test['ratio'] = test_cv_ratio

In [32]:
test[:3]

,X,X_length,ratio
1,gettonan,8,0.600000
2,incidentali,11,0.833333
3,involtino,9,0.800000


In [33]:
test[test.X_length.isnull()]
train[train.X_length.isnull()]

,X,form,word,base,ending,ratio,X_length,end_l


In [34]:
test[test.ratio.isnull()]
train[train.ratio.isnull()]

,X,form,word,base,ending,ratio,X_length,end_l


In [17]:
#%%time
all_data = pd.concat([train, test])
cv = CountVectorizer(analyzer='char_wb', lowercase = True, ngram_range=(1, 8))
X = cv.fit_transform(all_data.X)
X = csr_matrix(hstack([X, csr_matrix(all_data[['X_length', 'ratio']])]))
X_train, X_test = X[:len(train)], X[len(train):]
clf = LogisticRegression(C=50)
clf.fit(X_train, train.form)
form_pred = clf.predict(X_test)
print 'form predicted'

form predicted


In [13]:
all_data = pd.concat([train, test])
cv = CountVectorizer(analyzer='char_wb', lowercase = True, ngram_range=(1, 8))
X = cv.fit_transform(all_data.X)
X = csr_matrix(hstack([X, csr_matrix(all_data[['X_length', 'ratio']])]))
X_train, X_test = X[:len(train)], X[len(train):]
clf = LogisticRegression(C=50)
cross_val_score(clf, X_train, train.form)

array([ 0.96715384,  0.96601512,  0.96833987])

In [15]:
%%time
all_data = pd.concat([train, test])
cv = CountVectorizer(analyzer='char_wb', lowercase = True, ngram_range=(1, 10))
X = cv.fit_transform(all_data.X)
X = csr_matrix(hstack([X, csr_matrix(all_data[['X_length', 'ratio']])]))
X_train, X_test = X[:len(train)], X[len(train):]
clf = LogisticRegression(C=50)
print cross_val_score(clf, X_train, train.form)

[ 0.9665217   0.96515538  0.96793526]
Wall time: 3min 14s


In [18]:
%%time
test['form'] = form_pred
test = pd.get_dummies(test, columns=['form'])
train = pd.get_dummies(train, columns=['form'])
all_data = pd.concat([train, test])
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase = True)
X = cv.fit_transform(all_data.X)
X = csr_matrix(hstack([X, csr_matrix(all_data[['form_A','form_N','form_V', 'X_length', 'ratio']])]))
X_train, X_test = X[:len(train)], X[len(train):]
clf.fit(X_train, train['ending'])
ending_pred = clf.predict(X_test)
print 'ending predicted'

ending predicted
Wall time: 10min 1s


In [14]:
all_data = pd.concat([train, test])
clf = LogisticRegression(C=50)
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase = True)
X = cv.fit_transform(all_data.X)
X = csr_matrix(hstack([X, csr_matrix(all_data[['X_length', 'ratio']])]))
X_train, X_test = X[:len(train)], X[len(train):]
print cross_val_score(clf, X_train, train['ending'])

[ 0.95028922  0.95004806  0.95249665]


C:\Users\Petr\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [35]:
test['ending'] = ending_pred
all_data = pd.concat([train, test])

In [36]:
test[:3]

,X,X_length,ratio,ending
1,gettonan,8,0.600000,re
2,incidentali,11,0.833333,e
3,involtino,9,0.800000,are


In [37]:
v_or_c = []
for ending in all_data.ending:
    if len(ending) != 0:
        v_or_c.append(vowel_or_consonant(ending[::-1]))
    else:
        v_or_c.append(10)
all_data['v_or_c'] = v_or_c

In [38]:
all_data[:5]

,X,X_length,base,end_l,ending,form,ratio,word,v_or_c
1,vergognerete,12,vergogn,5.0,are,V,0.714286,vergognare,1
2,amnistiavate,12,amnistia,4.0,re,V,1.000000,amnistiare,0
3,menomazione,11,menomazione,0.0,,N,1.200000,menomazione,10
4,sfaldavamo,10,sfalda,4.0,re,V,0.666667,sfaldare,0
5,sfodererei,10,sfoder,4.0,are,V,1.000000,sfoderare,1


In [39]:
all_data = pd.get_dummies(all_data, columns=['ending'])
all_data = pd.get_dummies(all_data, columns=['form'])

In [40]:
pd.set_option('display.max_seq_items', len(all_data))
pd.set_option('display.max_columns', 500)

In [41]:
all_data.columns

Index([u'X', u'X_length', u'base', u'end_l', u'ratio', u'word', u'v_or_c',
       u'ending_', u'ending_a', u'ending_a-dati', u'ending_a-estate',
       u'ending_a-gol', u'ending_a-italia', u'ending_a-lavoro',
       u'ending_a-ordinanza', u'ending_a-paese', u'ending_a-paga',
       u'ending_a-partito', u'ending_adonna', u'ending_aforte', u'ending_an',
       u'ending_are', u'ending_arsi', u'ending_asorte', u'ending_buono',
       u'ending_cere', u'ending_dere', u'ending_desistere', u'ending_e',
       u'ending_e-quadro', u'ending_e-spia', u'ending_edere', u'ending_enere',
       u'ending_enire', u'ending_ere', u'ending_ernere', u'ending_essere',
       u'ending_ettere', u'ending_ge', u'ending_gere', u'ending_gge',
       u'ending_ggere', u'ending_gliere', u'ending_grande', u'ending_guere',
       u'ending_ia', u'ending_iare', u'ending_igere', u'ending_imere',
       u'ending_ingere', u'ending_io', u'ending_ire', u'ending_lere',
       u'ending_mpere', u'ending_ndere', u'ending_nere', u

In [43]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 10), lowercase = True, max_df=0.5)
X = cv.fit_transform(all_data.X)
print X.shape
X = csr_matrix(hstack([X, csr_matrix(all_data[['X_length', 'form_A', 'form_N', 'form_V',
                                               'ratio', 'v_or_c', u'ending_', u'ending_a', u'ending_a-dati',
                                               u'ending_a-estate', u'ending_a-gol', u'ending_a-italia',
                                               u'ending_a-lavoro', u'ending_a-ordinanza', u'ending_a-paese',
                                               u'ending_a-paga', u'ending_a-partito', u'ending_adonna',
                                               u'ending_aforte', u'ending_an', u'ending_are', u'ending_arsi',
                                               u'ending_asorte', u'ending_buono', u'ending_cere', u'ending_dere',
                                               u'ending_desistere', u'ending_e', u'ending_e-quadro', u'ending_e-spia',
                                               u'ending_edere', u'ending_enere', u'ending_enire', u'ending_ere',
                                               u'ending_ernere', u'ending_essere', u'ending_ettere', u'ending_ge',
                                               u'ending_gere', u'ending_gge', u'ending_ggere', u'ending_gliere',
                                               u'ending_grande', u'ending_guere', u'ending_ia', u'ending_iare',
                                               u'ending_igere', u'ending_imere', u'ending_ingere', u'ending_io',
                                               u'ending_ire', u'ending_lere', u'ending_mpere', u'ending_ndere',
                                               u'ending_nere', u'ending_nire', u'ending_o', u'ending_o-chiave',
                                               u'ending_o-leninismo', u'ending_o-radar', u'ending_o-stato',
                                               u'ending_oclan', u'ending_ogruppo', u'ending_olere', u'ending_olista',
                                               u'ending_omorta', u'ending_ompee', u'ending_ompere', u'ending_ondere',
                                               u'ending_opattuglia', u'ending_opraedere', u'ending_ore',
                                               u'ending_orire', u'ending_otere', u'ending_oviro', u'ending_re',
                                               u'ending_rere', u'ending_rire', u'ending_risedere', u'ending_rre',
                                               u'ending_rsi', u'ending_scere', u'ending_scuotere', u'ending_si',
                                               u'ending_ssr', u'ending_tere', u'ending_to', u'ending_ttere',
                                               u'ending_uocere', u'ending_uoere', u'ending_uotere', u'ending_uovere',
                                               u'ending_urre', u'ending_uscire', u'ending_vere', u'ending_x',
                                               u'ending_y']])]))
print X.shape
X_train, X_test = X[:len(train)], X[len(train):]
clf.fit(X_train, train['end_l'])
ending_len_pred = clf.predict(X_test)
print 'len predicted'

(148301, 1015965)
(148301, 1016062)
len predicted


In [15]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(1, 10), lowercase = True, max_df=0.5)
X = cv.fit_transform(all_data.X)
X = csr_matrix(hstack([X, csr_matrix(all_data[[u'X_length', u'ratio']])]))
X_train, X_test = X[:len(train)], X[len(train):]
print cross_val_score(clf, X_train, train['end_l'])
print 'len predicted'

[ 0.92705805  0.92608608  0.92954327]
len predicted


In [44]:
predictions = []
for i in np.linspace(1, len(ending_len_pred), len(ending_len_pred)).astype(int):
    cutted_word = test.X[i][:-ending_len_pred[i-1]] if ending_len_pred[i-1] > 0 else test.X[i]
    predictions.append(cutted_word + ending_pred[i-1] + '+' + form_pred[i-1])

In [45]:
predictions[:5]

['gettonare+V', 'incidentale+A', 'involtare+V', 'lievo+N', 'comunistizzare+V']

In [46]:
sample['Category'] = predictions
sample.to_csv('contest_3_submission8.csv')